<a href="https://colab.research.google.com/github/HaydenJF/CPSC392-Final-Project/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')
from plotnine import *

from sklearn.decomposition import PCA
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LogisticRegression
from sklearn.preprocessing import StandardScaler #Z-score variables
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split # simple TT split cv


import numpy as np

In [2]:
db1 = pd.read_csv("https://raw.githubusercontent.com/HaydenJF/CPSC392-Final-Project/main/CovidRates.csv")
db1.head()

,outcome,month,MMWR week,Age group,Vaccine product,Vaccinated with outcome,Fully vaccinated population,Unvaccinated with outcome,Unvaccinated population,Crude vax IR,Crude unvax IR,Crude IRR,Age adjusted vax IR,Age adjusted unvax IR,Age adjusted IRR,Continuity correction
0,case,APR 2021,202114,12-17,all_types,4,32950.0,27310,15994448.0,12.139605,170.746749,14.065263,NaN,NaN,NaN,0
1,case,APR 2021,202114,18-29,all_types,534,2354024.0,67219,28351677.0,22.684561,237.090032,10.451603,NaN,NaN,NaN,0
2,case,APR 2021,202114,30-49,all_types,1486,6849914.0,85688,37230385.0,21.693703,230.156094,10.609350,NaN,NaN,NaN,0
3,case,APR 2021,202114,50-64,all_types,1217,6800881.0,43509,20828144.0,17.894740,208.895233,11.673555,NaN,NaN,NaN,0
4,case,APR 2021,202114,65-79,all_types,1299,13424610.0,12391,6972605.0,9.676259,177.709766,18.365545,NaN,NaN,NaN,0


In [3]:
db2 = pd.read_csv("https://raw.githubusercontent.com/HaydenJF/CPSC392-Final-Project/main/CwithB.csv")
db2.head()

,outcome,month,mmwr_week,age_group,vaccine_product,boosted_with_outcome,boosted_population,primary_series_only_with_outcome,primary_series_only_population,unvaccinated_with_outcome,...,crude_primary_series_only_ir,crude_unvax_ir,crude_booster_irr,crude_irr,age_adj_booster_ir,age_adj_vax_ir,age_adj_unvax_ir,age_adj_booster_irr,age_adj_irr,continuity_correction
0,case,SEP 2021,202138,18-49,all_types,111,285893,41070,45089570,137013,...,91.085366,462.303877,11.907157,5.075501,NaN,NaN,NaN,NaN,NaN,0
1,case,SEP 2021,202138,50-64,all_types,123,366835,21505,25722971,37092,...,83.602318,451.806624,13.474673,5.404236,NaN,NaN,NaN,NaN,NaN,0
2,case,SEP 2021,202138,65+,all_types,220,842552,20403,24729783,19615,...,82.503757,473.343372,18.128018,5.737234,NaN,NaN,NaN,NaN,NaN,1
3,case,SEP 2021,202138,all_ages,all_types,454,1495280,82978,95542324,193720,...,86.849468,461.340983,15.194580,5.311961,35.590910,88.112335,462.062705,12.982605,5.244018,1
4,case,SEP 2021,202138,all_ages,Janssen,11,25805,8066,8186230,193720,...,98.531314,461.340983,10.822640,4.682176,47.514988,97.719144,462.062705,9.724567,4.728477,1


In [4]:
db3 = pd.read_csv("https://raw.githubusercontent.com/HaydenJF/CPSC392-Final-Project/main/casesperday.csv")
db3.head()

,State,Date,New Cases,7-Day Moving Avg,Historic Cases
0,United States,Apr 26 2022,63337,48692,582
1,United States,Apr 25 2022,53076,47243,2673
2,United States,Apr 24 2022,22536,45825,0
3,United States,Apr 23 2022,24035,45803,482
4,United States,Apr 22 2022,68140,44462,2538


In [6]:
print("\ndb1:")
print(*db1.columns, sep = "\n")
print("\ndb2:")
print(*db2.columns, sep = "\n")
print("\ndb3:")
print(*db3.columns, sep = "\n")


db1:
outcome
month
MMWR week
Age group
Vaccine product
Vaccinated with outcome
Fully vaccinated population
Unvaccinated with outcome
Unvaccinated population
Crude vax IR
Crude unvax IR
Crude IRR
Age adjusted vax IR
Age adjusted unvax IR
Age adjusted IRR
Continuity correction

db2:
outcome
month
mmwr_week
age_group
vaccine_product
boosted_with_outcome
boosted_population
primary_series_only_with_outcome
primary_series_only_population
unvaccinated_with_outcome
unvaccinated_population
crude_booster_ir
crude_primary_series_only_ir
crude_unvax_ir
crude_booster_irr
crude_irr
age_adj_booster_ir
age_adj_vax_ir
age_adj_unvax_ir
age_adj_booster_irr
age_adj_irr
continuity_correction

db3:
State
Date
New Cases
7-Day Moving Avg
Historic Cases


db1:
- outcome	
COVID-19 Case or Death [case; death]
Plain Text
- month	
Calendar month and year corresponding to MMWR week value [MMM YYYY format; e.g. APR 2021]
Plain Text
- MMWR week	
MMWR epidemiological year and week [YYYYWW format; e.g. 202101]
Plain Text
- Age group	
Age Group [5-11 years; 12-17 years; 18-29 years; 30-49 years; 50-64 years; 65-79 years; 80+ years; all_ages_adj]
Plain Text
- Vaccine product	
FDA-authorized COVID-19 vaccine product name [ Janssen; Moderna; Pfizer; all_types (includes Janssen, Moderna, Pfizer and unknown, FDA-approved vaccine product types)]
Plain Text
- Vaccinated with outcome	
Weekly count of individuals vaccinated with at least a primary series with the corresponding outcome
Number
- Fully vaccinated population	
Cumulative weekly count of the population vaccinated with at least a primary series
Number
- Unvaccinated with outcome	
Weekly count of unvaccinated individuals with the corresponding outcome
Number
- Unvaccinated population	
Cumulative weekly estimated count of the unvaccinated population
Number
- Crude vax IR	
Unadjusted incidence rate of the corresponding outcome among the population vaccinated with at least a primary series (per 100,000 population)
Number
- Crude unvax IR	
Unadjusted incidence rate of the corresponding outcome among the unvaccinated population (per 100,000 population)
Number
- Crude IRR	
Unadjusted incidence rate ratio (unvaccinated : vaccinated with at least a primary series)
Number
- Age adjusted vax IR	
Age-standardized incidence rate of the corresponding outcome among the population vaccinated with at least a primary series (per 100,000 population)
Number
- Age adjusted unvax IR	
Age-standardized incidence rate of the corresponding outcome among the unvaccinated population (per 100,000 population)
Number
- Age adjusted IRR	
Age-standardized incidence rate ratio (unvaccinated rate : vaccinated with at least a primary series rate)
Number
- Continuity correction	
Flag for whether continuity correction was applied for one or more jurisdictions in the strata [1=Yes, 0=No]. The continuity correction is applied to the denominators by capping the percent population coverage at 95%. To do this, we assumed that at least 5% of each age group would always be unvaccinated in each jurisdiction. Adding this correction ensures that there is always a reasonable denominator for the unvaccinated population and prevents incidence and death rates from growing unrealistically large due to potential overestimates of vaccination coverage.
Number

April 15th 2022

https://data.cdc.gov/Public-Health-Surveillance/Rates-of-COVID-19-Cases-or-Deaths-by-Age-Group-and/3rge-nu2a

db2:
- outcome	
COVID-19 Case or Death [case; death]
Plain Text
- month	
Calendar month and year corresponding to the MMWR week value [MMM YYYY format; e.g. SEP 2021]
Plain Text
- mmwr_week	
MMWR epidemiological year and week [YYYYWW format; e.g. 202101]
Plain Text
- age_group	
Age Group [12-17 years; 18-49 years; 50-64 years; 65+ years; all_ages_adj]
Plain Text
- vaccine_product	
FDA-authorized COVID-19 vaccine product name [ Janssen; Moderna; Pfizer; all_types (includes Janssen, Moderna, Pfizer and unknown, FDA-approved vaccine product types)]
Plain Text
- boosted_with_outcome	
Weekly count of individuals vaccinated with a primary series and booster dose with the corresponding outcome
Number
- boosted_population	
Cumulative weekly count of the population vaccinated with a primary series and booster dose
Number
- primary_series_only_with_outcome	
Weekly count of individuals vaccinated with a primary series only with the corresponding outcome
Number
- primary_series_only_population	
Cumulative weekly count of the population vaccinated with a primary series only
Number
- unvaccinated_with_outcome	
Weekly count of unvaccinated individuals with the corresponding outcome
Number
- unvaccinated_population	
Cumulative weekly estimated count of the unvaccinated population
Number
- crude_booster_ir	
Unadjusted incidence rate of the corresponding outcome among the population vaccinated with a primary series and booster dose (per 100,000 population)
Number
- crude_primary_series_only_ir	
Unadjusted incidence rate of the corresponding outcome among the population vaccinated with a primary series only (per 100,000 population)
Number
- crude_unvax_ir	
Unadjusted incidence rate of the corresponding outcome among the unvaccinated population (per 100,000 population)
Number
- crude_booster_irr	
Unadjusted incidence rate ratio (unvaccinated rate : vaccinated with a primary series and booster dose rate)
Number
- crude_irr	
Unadjusted incidence rate ratio (unvaccinated rate : vaccinated with a primary series only rate)
Number
- age_adj_booster_ir	
Age-standardized incidence rate of the corresponding outcome among the population vaccinated with a primary series and booster dose (per 100,000 population)
Number
- age_adj_vax_ir	
Age-standardized incidence rate of the corresponding outcome among the population vaccinated with a primary series only (per 100,000 population)
Number
- age_adj_unvax_ir	
Age-standardized incidence rate of the corresponding outcome among the unvaccinated population (per 100,000 population)
Number
- age_adj_booster_irr	
Age-standardized incidence rate ratio (unvaccinated rate : vaccinated with a primary series and booster dose rate)
Number
- age_adj_irr	
Age-standardized incidence rate ratio (unvaccinated rate : vaccinated with a primary series only rate)
Number
- continuity_correction	
Flag for whether continuity correction was applied for one or more jurisdictions in the strata [1=Yes, 0=No]. The continuity correction is applied to the denominators by capping the percent population coverage at 95%. To do this, we assumed that at least 5% of each age group would always be unvaccinated in each jurisdiction. Adding this correction ensures that there is always a reasonable denominator for the unvaccinated population and prevents incidence and death rates from growing unrealistically large due to potential overestimates of vaccination coverage.
Number

April 15th 2022

https://data.cdc.gov/Public-Health-Surveillance/Rates-of-COVID-19-Cases-or-Deaths-by-Age-Group-and/d6p8-wqjm











































db3:
- state (just says united states)
- date (shows the physical date) text but could become Number
- New Cases (number of new cases per day) Number
- 7-Day Moving Avg (average number of cases over 7 days) Number
- Historic Cases (cases demed to be recovered) Number

April 26th 2022

- Question 1: What are the main clusters of increased cases? (cluster model)
- Question 2: What is the case rate for unvaccinated, vaccinated and boostered individuals during each surge of cases?
- Question 3: What is the death rate for unvaccinated, vaccinated and boostered individuals during each surge of cases?
- Question 4: What is the case rate for each age group during each surge of cases?
- Question 5: What is the death rate for each age group during each surge of cases?
- Question 6: Can you create a model that predicts the date of maximum cases for each surge?  (supervised model)
- Question 7: What are the 3 most important variables (with the data given) to predict the deathrate for a given week? (instance of dimensionality reduction)

**ANALYSIS**

Covid Data (including cases, deaths w/ and w/out age)
- Question 1: What are the main clusters of increased cases? (cluster model)
 - **variables**: (only for cases)
mmwr_week
age_group
vaccine_product
boosted_with_outcome
boosted_population
primary_series_only_with_outcome
primary_series_only_population
unvaccinated_with_outcome
unvaccinated_population

   - all continous
 - **cleaning**: 
 would need to convert all variables into dummies of themselves (being as the variables are already semi-dummied it would be good to isolate each value. I.E. create new variable for age, boosted type, per week for every age and boosted type. (Already cleaned) z-score data
 - **modeling/computation**: 
This will be a train/validate/test EM model with a 60 20 20 split.  I am thinking because covid data already looks like a bell curve, an EM model will be very effective in modeling the data.
 - **graphs**: 
  The main graph I want to do is a cases vs. time graph, similar to the one you see on nytimes, and see how accurately it clusters the covid outbreaks.


- Question 6: Can you create a model that predicts the date of maximum cases for each surge?  (supervised model)
 - **variables**: (only for cases)
mmwr_week
age_group
vaccine_product
boosted_with_outcome
boosted_population
primary_series_only_with_outcome
primary_series_only_population
unvaccinated_with_outcome
unvaccinated_population

   - all continous
 - **cleaning**: 
 would need to convert all variables into dummies of themselves (being as the variables are already semi-dummied it would be good to isolate each value. I.E. create new variable for age, boosted type, per week for every age and boosted type. (Already cleaned) z-score data
 - **modeling/computation**: 
 I want to create two models.  I want to create a linear regression model using Leave one out cross validation.  I then want to create a model that tries to predict the maximum cases by using derivates (1st derivative goes to 0, 2nd derivate is negative).  I then want to compare their r^2 scores.
 - **graphs**: 
 I want to create an overlaying graph of my linear regression prediction vs. my derivative prediction vs. the actual values.


- Question 7: What are the 3 most important variables (with the data given) to predict the deathrate for a given week? (instance of dimensionality reduction)
 - **variables**: 
 (deaths and cases)
mmwr_week
age_group
vaccine_product
boosted_with_outcome
boosted_population
primary_series_only_with_outcome
primary_series_only_population
unvaccinated_with_outcome
unvaccinated_population
  - all continous (except deaths and cases which would need to be split)
 - **cleaning**: 
  would need to convert all variables into dummies of themselves (being as the variables are already semi-dummied it would be good to isolate each value. I.E. create new variable for age, boosted type, per week for every age and boosted type. (Already cleaned) z-score data.  Would also need to split data based on deaths and cases.
 - **modeling/computation**: 
 Would just create a linear regression model to predict death rates, then would use the 3 highest coefficients.  With those, I would create a new linear regression model (with just those variables) and see how well it does.  Both linear regressions would use Leave One Out cross validation.
 - **graphs**: 
 Overlay, deathrate with normal linear regression, death rate with 3 highest coefficient linear regression and actual deathrate.  